## 제목을 입력합니다

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pykrx import stock as st
import pickle

### csv 불러오기 실습

DataFrame : 2차원 데이터 -> table

In [2]:
close_model = pickle.load(open('predict_close_model_new.sav', 'rb'))
updown_model = pickle.load(open('predict_updown_model_new.sav', 'rb'))
date_label = pickle.load(open('date_label.sav', 'rb'))
stock_label = pickle.load(open('stock_label.sav', 'rb'))

# 1. 날짜 라벨인코더 업데이트 -> pickle로 저장하기 clear
def updateLabelEncoder(today):
    global date_label
    maxLabel = max(date_label['num'])
    todayDate = st.get_market_ohlcv(today, today, "005930").reset_index()

    for s_date in todayDate['날짜'].astype(str):
        if s_date not in date_label['s_date'].values:
            new_row = pd.DataFrame({'s_date': [s_date], 'num': [maxLabel + 1]})
            date_label = pd.concat([date_label, new_row], ignore_index=True)
    pickle.dump(date_label, open('date_label.sav', 'wb'))

# 2. 가져온 전일, 당일 시세 정보 전처리 : 가격예측&등락예측 모델용
def preprocessingNewData(today):
    updateLabelEncoder(today)
    columnMapper = {"티커":"isin", "시가" : "open", "고가" : "high", "저가" : "low", "종가" : "close", "거래량" : "volume", "거래대금" : "amount", "등락률" : "updown"}

    result = pd.DataFrame([])
    date_set = date_label.tail(2)['s_date'].str.replace("-",'').to_list()
    date_set_dash = date_label.tail(2)['s_date'].to_list()

    for i, date in enumerate(date_set):
        new = st.get_market_ohlcv(date).reset_index()
        new["s_date"] = date_set_dash[i]
        result = pd.concat([result,new])

    result.rename(columnMapper, axis=1, inplace=True)
    result = result[['isin', 's_date', 'open', 'high', 'low', 'close', 'volume', 'amount', 'updown']].sort_values(['isin', 's_date'])
    result['r_price'] = result['close'].shift(-1)
    result['r_updown'] = (0.5 * (result['updown'].shift(-1) / abs(result['updown'].shift(-1))) + 0.5)
    result.fillna(0, inplace= True)
    result['isin'] = result['isin'].replace(stock_label.set_index('isin')['num'])
    score = result[result["s_date"] != date_set_dash[-1]].reset_index(drop=True)
    predict = result[result["s_date"] == date_set_dash[-1]].reset_index(drop=True)
    score['s_date'] = score['s_date'].replace(date_label.set_index('s_date')['num'])
    predict['s_date'] = predict['s_date'].replace(date_label.set_index('s_date')['num'])
    r_price = score.pop("r_price")
    r_updown = score.pop("r_updown")
    predict.pop("r_price")
    predict.pop("r_updown")

    return [score, r_price, r_updown, predict]

# 3. 예측 결과 테이블 작성 및 json 반환
def predictToday(today):
    [_, r_price, r_updown, new_data] = preprocessingNewData(today)
    p_price = pd.DataFrame(close_model.predict(new_data), columns=['p_price'])
    e_updown = pd.DataFrame(updown_model.predict(new_data), columns=['e_updown'])

    result = pd.concat([new_data, p_price, e_updown], axis=1)
    result['p_price'] = round(result['p_price']).astype(int)
    result['e_updown'] = (round(result['e_updown'], 1) + 0.5).astype(int)
    result['p_rate'] = round((result['p_price'] - result['close']) / result['close'] * 100, 2)
    result['tmp_updown'] = (result['p_rate'] > 0).astype(int)
    result['predict'] = ((result['e_updown'] == 1) & (result['tmp_updown'] == 1)).astype(int)
    result.drop(['open', 'high', 'low', 'volume', 'amount', 'updown', "tmp_updown", 'e_updown'], axis =1, inplace=True)
    result['isin'].replace(stock_label.set_index('num')['isin'], inplace=True)
    result['s_date'].replace(date_label.set_index('num')['s_date'], inplace=True)
    return result

# 4. 예측 평가 테이블 작성 및 json 반환
def scoreYesterday(today):
    [new_data, r_price, r_updown, _] = preprocessingNewData(today)
    p_price = pd.DataFrame(close_model.predict(new_data), columns=['p_price'])
    e_updown = pd.DataFrame(updown_model.predict(new_data), columns=['e_updown'])

    result = pd.concat([new_data, p_price, r_price , e_updown, r_updown], axis=1)
    result['p_price'] = round(result['p_price']).astype(int)
    result['e_updown'] = (round(result['e_updown'], 1) + 0.5).astype(int)
    result['p_rate'] = round((result['p_price'] - result['close']) / result['close'] * 100, 2)
    result['r_rate'] = round((result['r_price'] - result['close']) / result['close'] * 100, 2)
    result['tmp_updown'] = (result['p_rate'] > 0).astype(int)
    result['predict'] = ((result['e_updown'] == 1) & (result['tmp_updown'] == 1)).astype(int)
    result['correct'] = (result['predict']==result['r_updown']).astype(int)
    result['error'] = round((abs((result['p_price'] - result['r_price']) / result['r_price']) * 100), 2)
    result.drop(['open', 'high', 'low', 'volume', 'amount', 'updown', 'tmp_updown', "e_updown","r_updown"], axis =1, inplace=True)
    result['isin'].replace(stock_label.set_index('num')['isin'], inplace=True)
    result['s_date'].replace(date_label.set_index('num')['s_date'], inplace=True)
    return result

In [3]:
date_label

,s_date,num
0,2013-01-02,0
1,2013-01-03,1
2,2013-01-04,2
3,2013-01-07,3
4,2013-01-08,4
...,...,...
2591,2023-07-12,2591
2592,2023-07-13,2592
2593,2023-07-14,2593
2594,2023-07-17,2594


In [4]:
today_date = "20230718"
updateLabelEncoder(today_date)
[score, r_price, r_updown, predict] = preprocessingNewData(today_date)

In [5]:
score = scoreYesterday("20230711")
predict = predictToday("20230710")

In [6]:
score

,isin,s_date,close,p_price,r_price,p_rate,r_rate,predict,correct,error
0,000020,2023-07-17,10280,10298,10000.0,0.18,-2.72,0,1,2.98
1,000040,2023-07-17,536,550,519.0,2.61,-3.17,1,0,5.97
2,000050,2023-07-17,9770,9791,9640.0,0.21,-1.33,0,1,1.57
3,000070,2023-07-17,71700,71674,71200.0,-0.04,-0.70,0,1,0.67
4,000075,2023-07-17,54000,54001,53500.0,0.00,-0.93,0,1,0.94
...,...,...,...,...,...,...,...,...,...,...
943,457190,2023-07-17,341500,340740,341500.0,-0.22,0.00,0,1,0.22
944,460850,2023-07-17,9800,9803,9460.0,0.03,-3.47,1,0,3.63
945,460860,2023-07-17,9710,9712,9410.0,0.02,-3.09,1,0,3.21
946,900140,2023-07-17,2570,2573,2580.0,0.12,0.39,0,0,0.27


In [13]:
predict['s_date'] = predict['s_date'].str.replace("-","")
predict

,isin,s_date,close,p_price,p_rate,predict
0,000020,20230718,10000,9993,-0.07,0
1,000040,20230718,519,527,1.54,1
2,000050,20230718,9640,9647,0.07,0
3,000070,20230718,71200,71144,-0.08,0
4,000075,20230718,53500,53495,-0.01,0
...,...,...,...,...,...,...
943,457190,20230718,341500,341271,-0.07,0
944,460850,20230718,9460,9452,-0.08,0
945,460860,20230718,9410,9404,-0.06,0
946,900140,20230718,2580,2592,0.47,0


In [42]:
score = pd.concat([pd.read_csv("predictTableSample.csv", dtype=(str)), score])

In [40]:
preidctSample = pd.concat([score, predict])
preidctSample

,isin,s_date,close,p_price,r_price,p_rate,r_rate,predict,correct,error
0,000020,2023-07-07,10100,10098,9840.0,-0.02,-2.57,0,1,2.62
1,000040,2023-07-07,554,560,558.0,1.08,0.72,0,0,0.36
2,000050,2023-07-07,9840,9860,9470.0,0.2,-3.76,0,1,4.12
3,000070,2023-07-07,71000,70960,70100.0,-0.06,-1.27,0,1,1.23
4,000075,2023-07-07,53100,53049,53500.0,-0.1,0.75,0,0,0.84
...,...,...,...,...,...,...,...,...,...,...
948,457190,2023-07-11,315000,314856,NaN,-0.05,NaN,0,NaN,NaN
949,460850,2023-07-11,10060,10075,NaN,0.15,NaN,0,NaN,NaN
950,460860,2023-07-11,9770,9781,NaN,0.11,NaN,0,NaN,NaN
951,900140,2023-07-11,2620,2653,NaN,1.26,NaN,0,NaN,NaN


In [15]:
predict

,isin,s_date,close,p_price,p_rate,predict
0,000020,2023-07-11,9930,9940,0.10,0
1,000040,2023-07-11,557,579,3.95,0
2,000050,2023-07-11,9510,9530,0.21,0
3,000070,2023-07-11,70400,70379,-0.03,0
4,000075,2023-07-11,53500,53483,-0.03,0
...,...,...,...,...,...,...
948,457190,2023-07-11,315000,314856,-0.05,0
949,460850,2023-07-11,10060,10075,0.15,0
950,460860,2023-07-11,9770,9781,0.11,0
951,900140,2023-07-11,2620,2653,1.26,0


In [27]:
predict[predict['predict'] == 1].sort_values('p_rate')

,isin,s_date,close,p_price,p_rate,predict
41,000660,2023-07-11,113400,113408,0.01,1
458,012320,2023-07-11,88300,88312,0.01,1
60,000995,2023-07-11,26050,26055,0.02,1
241,004545,2023-07-11,14520,14526,0.04,1
251,004835,2023-07-11,5110,5112,0.04,1
204,003620,2023-07-11,8620,8624,0.05,1
669,069460,2023-07-11,1340,1341,0.07,1
101,001570,2023-07-11,105900,106007,0.10,1
288,005720,2023-07-11,4325,4330,0.12,1
300,005930,2023-07-11,71500,71594,0.13,1
